### SEVER DEFINITIONS


In [2]:
# Necessary Imports
from flask import Flask, jsonify
from werkzeug.routing import IntegerConverter
from _csv import Dialect as _Dialect
import csv

from rtls_single_slave import aoa_single_main, pixel_calculate
from rtls_multiple_slave import aoa_multiple_main



# Handle Negative Values
class SignedIntConverter(IntegerConverter):
    regex = r'-?\d+'
app = Flask(__name__)
app.url_map.converters['signed_int'] = SignedIntConverter
#Store input values: AoA and Horizontal Angle

aoabefore = [1]
aoa= [[] for i in range(1)]
lev = []

@app.route('/')
def home():
    return "This is the HomePage"

#Routing Settings
@app.route('/send/<signed_int:horiz>', methods=['GET'])
def get_angle(horiz):
    lev.append(horiz)
    print("Plane Angle: " + str(horiz))
    aoabefore.clear()
    aoabefore.extend(aoa.copy())

    aoa.clear()
    yy = []
    aoa.extend(aoa_single_main())
    return jsonify({'state': 'success'})

@app.route('/calculate/', methods=['GET'])
def calculate():
    print("Before Turning："+str(aoabefore)+'\n'+"After Turning： " + str(aoa) +'\n'+"Rotation Angle： "+ str(lev))
    #aoa_image()
    u = []
    v = []
    minnumber = min(len(aoabefore), len(aoa))
    f = open('./aoa_results/aoa_data_after_calibration.csv','a',newline="")
    f.truncate()
    writer = csv.writer(f)
    

    for i in range(minnumber):
        for j in range(minnumber):
            (temp1, temp2) = pixel_calculate(lev[-2] - lev[-1], aoabefore[i], aoa[j])

            if (temp1 != 1):
                u.append(temp1)
                v.append(temp2)
                tupx = (temp1,temp2)
                writer.writerow(tupx)
    
    f.close()
    
    return jsonify({'state': 'success'})

### RUNNING THE SEVER

In [4]:
app.run(host='0.0.0.0',port = 5000, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.19.217.212:5000
Press CTRL+C to quit


Plane Angle: 4


[2023-03-09 15:42:45,876] ERROR in app: Exception on /send/4 [GET]
Traceback (most recent call last):
  File "d:\Python_VENV\rtls_AoA\lib\site-packages\flask\app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
  File "d:\Python_VENV\rtls_AoA\lib\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "d:\Python_VENV\rtls_AoA\lib\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "d:\Python_VENV\rtls_AoA\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\sanji\AppData\Local\Temp\ipykernel_4408\1592888128.py", line 37, in get_angle
    aoa.extend(aoa_single_main())
  File "e:\intern_VisBLE\VisBLE_INTERN\AoA Point Cloud\rtls_single_slave.py", line 271, in aoa_single_main
    return main1()
  File "e:\intern_VisBLE\VisBLE_INTERN\AoA Point Cloud\rtls_sing

=== ERROR: No one of the nodes identified as RTLS MASTER ===
Try to stop AOA result parsing thread
AOA Stopped
Done


10.19.190.195 - - [09/Mar/2023 15:42:47] "GET /calculate/ HTTP/1.1" 200 -


Before Turning：[[]]
After Turning： []
Rotation Angle： [4]
